In [35]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def collect_data(course_urls_file):
    # Initialize lists to store data
    sku_list = []
    name_list = []
    price_list = []
    in_stock_list = []
    categories_list = []
    tags_list = []

    # Read the links from the file
    with open(course_urls_file, 'r') as file:
        urls = file.read().splitlines()

    # Loop through the URLs and scrape data
    for url in urls:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            

            # Extract data from the webpage
            sku = soup.find('span', class_='sku').text.strip()
            print(sku)
            
            name = soup.find('h1').text.strip()
            
            #price = soup.find('h1', class_='price').text.strip().split()[1]
            price_element = soup.find('p', class_='price')
            span_contents = price_element.find('span', class_='woocommerce-Price-amount').contents
            price = span_contents[1].strip()
            
            in_stock = int(soup.find('p', class_='stock').text.strip().split()[0])
            
            categories = soup.find('span', class_='posted_in').text.strip().split(': ')[1]
            print(categories)
            
            
            tags = soup.find('span', class_='tagged_as').text.strip().split(': ')[1]
            print(tags)
            

            # Append data to lists
            sku_list.append(sku)
            name_list.append(name)
            price_list.append(price)
            in_stock_list.append(in_stock)
            categories_list.append(categories)
            tags_list.append(tags)

    # Create a DataFrame from the collected data
    data = {
        'SKU': sku_list,
        'Name': name_list,
        'Price': price_list,
        'InStock': in_stock_list,
        'Categories': categories_list,
        'Tags': tags_list
    }
    df = pd.DataFrame(data)

    return df

# Usage example:
if __name__ == "__main__":
    course_urls_file = "pokemon1.txt"
    pokemon_data = collect_data(course_urls_file)
    pokemon_data.to_csv("pokemon_data.csv", index=False)


4391
Pokemon, Seed
bulbasaur, Overgrow, Seed


In [6]:
import time
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [7]:
BASE_URL = 'http://api.worldbank.org/v2/'
COUNTRIES = ["US", "IN", "CN", "JP", "CA", "GB", "ZA"]

INDICATORS = ['SP.POP.TOTL',
             'SP.POP.TOTL.FE.IN',
             'SP.POP.TOTL.MA.IN',
             'SP.DYN.CBRT.IN',
             'SP.DYN.CDRT.IN',
             'SP.DYN.LE00.MA.IN',
             'SP.DYN.LE00.FE.IN',
             'SE.PRM.ENRR',
             'SE.TER.ENRR',
             'SE.PRM.CMPT.ZS',
             'SE.ADT.1524.LT.ZS']

#TODO (option)
# If you need other initializations

In [41]:

def collect_data(countryCode, per_page, start_year, end_year):
    
    
    # Initialize lists to store data
    total_population = []
    female_population = []
    male_population = []
    birth_rate = []
    death_rate = []
    male_life_expectancy = []
    female_life_expectancy = []
    school_enrollment_primary = []
    school_enrollment_tertiary = []
    primary_completion_rate = []
    literacy_rate = []
    year = []
    country = []
    
    

    for indicator in INDICATORS:
        # Construct the API URL for the specific country and indicator
        url = f"{BASE_URL}country/{countryCode}/indicator/{indicator}?format=json&per_page={per_page}&date={start_year}:{end_year}"

        values = []  # Danh sách để lưu trữ các giá trị "value"    
            
            
        # Make a GET request to the World Bank API
        response = requests.get(url)

        if response.status_code == 200:
            # Convert the JSON response to a list of dictionaries
            response_data = response.json()
            if response_data:
                # Extract the relevant data from the response
                data_entries = response_data[1]
                for entry in data_entries:
                    if 'value' in entry and entry['value'] is not None:
                        value = entry['value']
                    else:
                        value = None
                    values.append(value)
                
                #print(len(values))
                
                # Dựa vào chỉ số hiện tại, xác định danh sách nào sẽ lưu giá trị
                if indicator == 'SP.POP.TOTL':
                    total_population = values
                elif indicator == 'SP.POP.TOTL.FE.IN':
                    female_population = values
                elif indicator == 'SP.POP.TOTL.MA.IN':
                    male_population = values
                elif indicator == 'SP.DYN.CBRT.IN':
                    birth_rate = values
                elif indicator == 'SP.DYN.CDRT.IN':
                    death_rate = values
                elif indicator == 'SP.DYN.LE00.MA.IN':
                    male_life_expectancy = values
                elif indicator == 'SP.DYN.LE00.FE.IN':
                    female_life_expectancy = values
                elif indicator == 'SE.PRM.ENRR':
                    school_enrollment_primary = values
                elif indicator == 'SE.TER.ENRR':
                    school_enrollment_tertiary = values
                elif indicator == 'SE.PRM.CMPT.ZS':
                    primary_completion_rate = values
                elif indicator == 'SE.ADT.1524.LT.ZS':
                    literacy_rate = values
    
    
    #===========================================================================================================
    # lấy danh danh các năm cho vào mảng year
    # Construct the API URL for the specific country and indicator
    url = f"{BASE_URL}country/{countryCode}/indicator/{INDICATORS[0]}?format=json&per_page={per_page}&date={start_year}:{end_year}"       
    # Make a GET request to the World Bank API
    response = requests.get(url)
    if response.status_code == 200:
        # Convert the JSON response to a list of dictionaries
        response_data = response.json()
        if response_data:
            # Extract the relevant data from the response
            data_entries = response_data[1]
            for entry in data_entries:
                # Lấy giá trị "value" từ mỗi dòng và thêm vào danh sách
                date = entry['date']
                year.append(date)
                
                country_temp = entry['countryiso3code']
                country.append(country_temp)
                
    #===========================================================================================================  

    
    # Create a DataFrame from the collected data
    data = {
        'Total Population': total_population,
        'Female Population': female_population,
        'Male Population': male_population,
        'Birth Rate': birth_rate,
        'Death Rate': death_rate,
        'Male life expectancy': male_life_expectancy,
        'Female life expectancy': female_life_expectancy,
        'School enrollment, primary': school_enrollment_primary,
        'School enrollment, tertiary': school_enrollment_tertiary,
        'Primary completion rate': primary_completion_rate,
        'Literacy rate': literacy_rate,
        'Year': year,
        'Country': country
    }
    
    df = pd.DataFrame(data)
    # Thay thế giá trị None bằng 'None' và giá trị NaN bằng giá trị rỗng
    df = df.fillna('None')

    return df

# # Example usage
# countryCode = "US"
# per_page = 100
# start_year = 2000
# end_year = 2003
# result = collect_data(countryCode, per_page, start_year, end_year)

# # Print the collected data
# print(result)


In [42]:
def Generate_Countries_Dataset(countryCode_list):
    data = pd.DataFrame()
    for countryCode in countryCode_list:
        data = pd.concat([data, collect_data(countryCode = countryCode, per_page = 100, start_year = 2000, end_year = 2003)], axis=0)
    return data


In [43]:
#TEST
data_countries = Generate_Countries_Dataset(COUNTRIES)

print(data_countries)
# assert data_countries.shape == (161, 13)

   Total Population  Female Population  Male Population  Birth Rate  \
0         290107933          147242226        142865707      14.100   
1         287625193          145972802        141652391      14.000   
2         284968955          144625822        140343133      14.100   
3         282162411          143251716        138910694      14.400   
0        1117415123          539178882        578236241      25.375   
1        1098313039          529978166        568334873      26.082   
2        1078970907          520679574        558291332      26.728   
3        1059633675          511410095        548223581      27.001   
0        1288400000          629369651        659030348      12.410   
1        1280400000          625534993        654865007      12.860   
2        1271850000          621436557        650413443      13.380   
3        1262645000          617082063        645562937      14.030   
0         127718000           65270637         62447363       8.900   
1     

In [44]:
# Save to csv file with name coutries.csv
#TODO
data_countries.to_csv("coutries1.csv", index=False, na_rep='') 

In [40]:
import pandas as pd

# Tạo DataFrame ví dụ
data = pd.DataFrame({
    'Column1': [1, None, 3, 4],
    'Column2': [None, 7, None, 9]
})

# Thay thế giá trị None bằng 'None' và giá trị NaN bằng giá trị rỗng
data = data.fillna('None')

# Ghi DataFrame vào tệp CSV
data.to_csv('output.csv', index=False, na_rep='')

# Đọc tệp CSV để kiểm tra kết quả
loaded_data = pd.read_csv('output.csv')
print(loaded_data)


  Column1 Column2
0     1.0    None
1    None     7.0
2     3.0    None
3     4.0     9.0
